In [1]:
import os
import re
import pickle
import numpy as np
import pandas as pd

# Temperature

In [111]:
basepath = 'temp_test'
savepath = 'datasci_dataset_2022'

location = ["BKK", "Chiangmai", "Khonkaen", "Rayong", "Saraburi", "Surat"]

In [112]:
corrupt_temp = []
for loc in location:
  loc_path = os.path.join(basepath, loc)
  output_path = os.path.join(savepath, loc)
  output_path = os.path.join(output_path, 'test')
  
  li = []
  for f in os.listdir(loc_path):
    df = pd.read_csv(os.path.join(loc_path, f))
    li.append(df)
  frame = pd.concat(li)
  frame['temp'] = frame['temp'].apply(lambda x: re.sub('[^0-9.]+', '', x)).astype(float)
  frame = frame.drop_duplicates()
  frame['date_time'] = pd.to_datetime(frame['date_time'])
  temp_df = frame.set_index('date_time').sort_index().loc['2020-07-01':'2021-07-01']
  
  group_by_temp = temp_df.groupby([temp_df.index.date]).count()
  print("Location:", loc)
  print(temp_df.shape)
  corrupt_temp.append(group_by_temp.loc[group_by_temp['temp'].between(9, 23, 'both')].index.to_series())
  print()
  print(output_path)
  temp_df.to_csv(os.path.join(output_path, f"{loc.lower()}_temp_surface.csv"))

Location: BKK
(5835, 1)

datasci_dataset_2022\BKK\test
Location: Chiangmai
(5835, 1)

datasci_dataset_2022\Chiangmai\test
Location: Khonkaen
(5835, 1)

datasci_dataset_2022\Khonkaen\test
Location: Rayong
(5835, 1)

datasci_dataset_2022\Rayong\test
Location: Saraburi
(5835, 1)

datasci_dataset_2022\Saraburi\test
Location: Surat
(5835, 1)

datasci_dataset_2022\Surat\test


In [90]:
corrupt_temp = pd.concat(corrupt_temp).drop_duplicates()

In [91]:
corrupt_temp

2021-01-01    2021-01-01
dtype: object

In [92]:
corrupt_temp.to_csv('corrupt_temp.csv', index=False)

# Wind

In [117]:
basepath = 'testThread'
savepath = 'datasci_dataset_2022'

location = ["BKK", "Chiangmai", "Khonkaen", "Rayong", "Saraburi", "Surat"]

In [118]:
corrupt_wind = []
for loc in location:
  loc_path = os.path.join(basepath, loc)
  output_path = os.path.join(savepath, loc)
  output_path = os.path.join(output_path, 'test')
  
  li = []
  for f in os.listdir(loc_path):
    df = pd.read_csv(os.path.join(loc_path, f))
    li.append(df)
  frame = pd.concat(li)
  frame = frame.drop_duplicates()
  frame['date_time']= pd.to_datetime(frame['date_time'])
  frame = frame.set_index('date_time').sort_index().loc['2020-07-01':'2021-07-01']
  
  group_by_wind = frame.groupby([frame.index.date]).count()
  print("Location:", loc)
  print(frame.shape)
  corrupt_wind.append(group_by_wind.loc[np.logical_or((group_by_wind['wind speed'].between(9, 23, 'both')),
                                        (group_by_wind['wind direction'].between(9, 23, 'both')))].index.to_series())
  print()
  print(output_path)
  frame.to_csv(os.path.join(output_path, f"{loc.lower()}_weather_wind.csv"))

Location: BKK
(5835, 2)

datasci_dataset_2022\BKK\test
Location: Chiangmai
(5835, 2)

datasci_dataset_2022\Chiangmai\test
Location: Khonkaen
(5835, 2)

datasci_dataset_2022\Khonkaen\test
Location: Rayong
(5835, 2)

datasci_dataset_2022\Rayong\test
Location: Saraburi
(5835, 2)

datasci_dataset_2022\Saraburi\test
Location: Surat
(5835, 2)

datasci_dataset_2022\Surat\test


In [95]:
wind_df = frame

In [96]:
corrupt_wind = pd.concat(corrupt_wind).drop_duplicates()

In [97]:
corrupt_wind

2021-01-01    2021-01-01
dtype: object

In [98]:
corrupt_wind.to_csv('corrupt_wind.csv', index=False)

In [99]:
wind_df

,wind speed,wind direction
date_time,,
2020-07-01 01:00:00,21.0,255.0
2020-07-01 04:00:00,29.0,255.0
2020-07-01 07:00:00,35.0,250.0
2020-07-01 10:00:00,36.0,250.0
2020-07-01 13:00:00,29.0,255.0
...,...,...
2021-07-01 19:00:00,9.0,245.0
2021-07-01 20:00:00,9.0,245.0
2021-07-01 21:00:00,9.0,235.0


# Merge Wind and Temperature

In [100]:
merge_df = temp_df.join(wind_df, on='date_time')

In [101]:
merge_df.loc[merge_df.isna().sum(axis=1).astype(bool)].sort_index()

,temp,wind speed,wind direction
date_time,,,


In [143]:
merge_df.head(100)

,temp,wind speed,wind direction
date_time,,,
2020-07-01 01:00:00,25.2,21.0,255.0
2020-07-01 04:00:00,24.7,29.0,255.0
2020-07-01 07:00:00,25.4,35.0,250.0
2020-07-01 10:00:00,29.3,36.0,250.0
2020-07-01 13:00:00,31.7,29.0,255.0
...,...,...,...
2020-07-12 22:00:00,25.1,31.0,265.0
2020-07-13 01:00:00,24.9,32.0,270.0
2020-07-13 04:00:00,24.2,34.0,270.0


In [102]:
merge_df.groupby([merge_df.index.date]).count().to_csv('testtest.csv')

In [103]:
merge_df

,temp,wind speed,wind direction
date_time,,,
2020-07-01 01:00:00,25.2,21.0,255.0
2020-07-01 04:00:00,24.7,29.0,255.0
2020-07-01 07:00:00,25.4,35.0,250.0
2020-07-01 10:00:00,29.3,36.0,250.0
2020-07-01 13:00:00,31.7,29.0,255.0
...,...,...,...
2021-07-01 19:00:00,26.8,9.0,245.0
2021-07-01 20:00:00,26.2,9.0,245.0
2021-07-01 21:00:00,25.9,9.0,235.0


In [104]:
merge_df.groupby([merge_df.index.date]).count()

,temp,wind speed,wind direction
2020-07-01,8,8,8
2020-07-02,8,8,8
2020-07-03,8,8,8
2020-07-04,8,8,8
2020-07-05,8,8,8
...,...,...,...
2021-06-27,24,24,24
2021-06-28,24,24,24
2021-06-29,24,24,24
2021-06-30,24,24,24
